<a href="https://colab.research.google.com/github/GabMer/Inteligencia-Artificial/blob/main/Enfoques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

11 de agosto del 2025

In [ ]:
# Ejemplo de EnfoqueSimbolico (GOFAI)
 #Sistema experto medico basicocon reglas logicas explicitas
def diagnosticar(sintomas):
  if "fiebre" in sintomas and "tos" in sintomas and "dolor_garganta" in sintomas:
    return "posible gripe"
  elif "fiebre" in sintomas and "dolor_cabeza" in sintomas and "rigidez_cuello" in sintomas:
    return "posible meningitis"
  elif "estornudos" in sintomas and "congestion" in sintomas:
    return "posible resfriado comun"
  elif "dolor_estomago" in sintomas and "nauseas" in sintomas:
    return "posible intoxicacion alimentaria"
  else:
    return "diagnostico no determinado"


paciente_1 = ["fiebre", "tos", "dolor_garganta"]
paciente_2 = ["fiebre", "dolor_cabeza", "rigidez_cuello"]
paciente_3 = ["estornudos", "congestion"]

print("paciente 1: ", diagnosticar(paciente_1))
print("paciente 2: ", diagnosticar(paciente_2))
print("paciente 3: ", diagnosticar(paciente_3))

paciente 1:  posible gripe
paciente 2:  posible meningitis
paciente 3:  posible resfriado comun


**Naive Bayes**

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix #falsos negativos

# Cargar el conjunto de datos de cáncer de mama
X,y = load_breast_cancer(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

nb= GaussianNB()

# Entrenar el clasificador
nb.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = nb.predict(X_test)

print("Naive Bayes - accuracy:", accuracy_score(y_test, y_pred))
print("Naive Bayes - confusion matrix:\n", confusion_matrix(y_test, y_pred))
print("Naive Bayes - classification report:\n", classification_report(y_test, y_pred))
#

Naive Bayes - accuracy: 0.9370629370629371
Naive Bayes - confusion matrix:
 [[46  7]
 [ 2 88]]
Naive Bayes - classification report:
               precision    recall  f1-score   support

           0       0.96      0.87      0.91        53
           1       0.93      0.98      0.95        90

    accuracy                           0.94       143
   macro avg       0.94      0.92      0.93       143
weighted avg       0.94      0.94      0.94       143



In [ ]:
#Clasificacion con Red NEuronal (MLP) en el mismo dataset
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

mlp= make_pipeline(
    StandardScaler(),
    MLPClassifier(hidden_layer_sizes=(32,16),
                  activation="relu",
                  solver="adam",
                  max_iter=500,
                  random_state=42)
)

mlp.fit(X_train, y_train)

y_pred = mlp.predict(X_test)

print("MLP - accuracy:", accuracy_score(y_test, y_pred))
print (confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(classification_report(y_test, y_pred))


MLP - accuracy: 0.965034965034965
[[52  1]
 [ 4 86]]
              precision    recall  f1-score   support

           0       0.93      0.98      0.95        53
           1       0.99      0.96      0.97        90

    accuracy                           0.97       143
   macro avg       0.96      0.97      0.96       143
weighted avg       0.97      0.97      0.97       143

              precision    recall  f1-score   support

           0       0.93      0.98      0.95        53
           1       0.99      0.96      0.97        90

    accuracy                           0.97       143
   macro avg       0.96      0.97      0.96       143
weighted avg       0.97      0.97      0.97       143



In [ ]:
#Programacion Genetica
!pip install deap --quiet #para instalar la libreria de aptitud

import operator, random
import numpy as np
from deap import algorithms, base, creator, tools, gp
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 1) Dataset real
X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

# 2) Definición de operadores básicos
pset = gp.PrimitiveSet("MAIN", X.shape[1])
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(np.sin, 1)
pset.addPrimitive(np.cos, 1)
pset.addEphemeralConstant("rand", lambda: random.uniform(-1, 1))

# 3) Definición de tipos de individuo y fitness
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)

toolbox= base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# 4) Evaluacion usamos accuracy como fitness
def eval_individual(individual):
    func = toolbox.compile(expr=individual)
    preds = [1 if func(*x) > 0 else 0 for x in X_train]
    return accuracy_score(y_train, preds),

toolbox.register("compile", gp.compile, pset=pset)
toolbox.register("evaluate", eval_individual)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr, pset=pset)
toolbox.register("select", tools.selTournament, tournsize=3)

# 5) Evolucion
pop = toolbox.population(n=50)
for gen in range(10):
    offspring = tools.selTournament(pop, len(pop), tournsize=3)
    offspring = list(map(toolbox.clone, offspring))
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < 0.5:
            toolbox.mate(child1, child2)
    for mutant in offspring:
        if random.random() < 0.2:
            toolbox.mutate(mutant)
    for ind in offspring:
        ind.fitness.values = toolbox.evaluate(ind)
    pop[:]= offspring
    print(f"Generacion {gen}: Mejor fitness = {max(ind.fitness.values[0] for ind in pop)}")

# 6) Mejor individuo y prueba
best_ind = tools.selBest(pop, k=1)[0]
func = toolbox.compile(expr=best_ind)
y_pred = [1 if func(*x) > 0 else 0 for x in X_test]
print("\nAccuracy en test:", accuracy_score(y_test, y_pred))
print("\n MEjor formula encontrada:", best_ind)

/usr/local/lib/python3.11/dist-packages/deap/gp.py:257: RuntimeWarning: Ephemeral rand function cannot be pickled because its generating function is a lambda function. Use functools.partial instead.
  warnings.warn("Ephemeral {name} function cannot be "
/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Generacion 0: Mejor fitness = 0.7370892018779343
Generacion 1: Mejor fitness = 0.7370892018779343
Generacion 2: Mejor fitness = 0.7370892018779343
Generacion 3: Mejor fitness = 0.7488262910798122
Generacion 4: Mejor fitness = 0.7370892018779343
Generacion 5: Mejor fitness = 0.7488262910798122
Generacion 6: Mejor fitness = 0.755868544600939
Generacion 7: Mejor fitness = 0.7769953051643192
Generacion 8: Mejor fitness = 0.7793427230046949
Generacion 9: Mejor fitness = 0.7793427230046949

Accuracy en test: 0.7342657342657343

 MEjor formula encontrada: mul(cos(ARG10), sub(ARG8, ARG27))


Programacion genetica, usa el concepto de mutacion, busca distintos caminos para ser mas exacto, pasa de generacion en generacion, buscando cual es el mejor de los caminos.